In [ ]:
import requests
import pandas as pd
from bokeh.layouts import column, row
from prophet import Prophet
from bokeh.plotting import figure, show
from bokeh.models import BoxZoomTool, WheelZoomTool, ResetTool, PanTool, SaveTool, BoxSelectTool, TapTool, CrosshairTool 
from bokeh.io import output_notebook
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import Select, TextInput, MultiSelect, CheckboxGroup, Checkbox

In [ ]:
# 1. Fetch BTCUSDT data from public endpoint for the last 10 days
url = "https://api.binance.com/api/v3/klines"
symbol = "BTCUSDT"
interval = "1d"
limit = 1000

params = {
    "symbol": symbol,
    "interval": interval,
    "limit": limit
}

response = requests.get(url, params=params)
data = response.json()

# Convert the data to a DataFrame
df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

# 2. Use Prophet python library to create a demand forecasting model
prophet_df = df[["timestamp", "close", "volume"]].rename(columns={"timestamp": "ds", "close": "y", "volume": "volume"})
prophet_df["y"] = pd.to_numeric(prophet_df["y"])
prophet_df["volume"].fillna(0, inplace=True)
prophet_df["volume"] = pd.to_numeric(prophet_df["volume"])

model = Prophet()
model.add_regressor("volume")

model.fit(prophet_df)

# Make future predictions
future = model.make_future_dataframe(periods=15)
future["volume"] = prophet_df["volume"] 
future["volume"].fillna(0, inplace=True)
forecast = model.predict(future)

# 3. Plot the price data and the forecasted price on the same plot using Bokeh
output_notebook()

source_actual = ColumnDataSource(prophet_df)
source_forecast = ColumnDataSource(forecast)

hover = HoverTool(tooltips=[("Date", "@ds{%F}"), ("Actual Price", "@y"), ("Forecasted Price", "@yhat")], formatters={"@ds": "datetime"})


def update_data(attrname, old, new):
    symbol = symbol_select.value
    interval = interval_select.value
    periods = predict_period_input.value
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    p.title.text = f"{symbol} Price Forecast"

    response = requests.get(url, params=params)
    data = response.json()

    model = Prophet(seasonality_mode=seasonality_mode_select.value, changepoint_prior_scale=float(changepoint_prior_scale_input.value))
    df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

    if use_volume_checkbox.active:
        prophet_df = df[["timestamp", "close", "volume"]].rename(columns={"timestamp": "ds", "close": "y", "volume": "volume"})
        prophet_df["volume"] = pd.to_numeric(prophet_df["volume"])
        model.add_regressor("volume")
    else:
        prophet_df = df[["timestamp", "close"]].rename(columns={"timestamp": "ds", "close": "y"})
    
    prophet_df["y"] = pd.to_numeric(prophet_df["y"])
    model.fit(prophet_df)
    future = model.make_future_dataframe(int(periods))
    future["volume"] = prophet_df["volume"] 
    future["volume"].fillna(0, inplace=True)
    forecast = model.predict(future)

    source_actual.data = prophet_df
    source_forecast.data = forecast

# UI Elements
predict_period_input = TextInput(value="5", title="Prediction:")
predict_period_input.on_change("value", update_data)

symbol_select = TextInput(value="BTCUSDT", title="Symbol:")
symbol_select.on_change("value", update_data)

interval_select = Select(title="Interval", options=["1d", "3d", "1w"], value="3d")
interval_select.on_change("value", update_data)

seasonality_mode_select = Select(title="Seasonality Mode", options=["additive", "multiplicative"], value="additive")
seasonality_mode_select.on_change("value", update_data)

changepoint_prior_scale_input = TextInput(value="0.05", title="Changepoint Prior Scale:")
changepoint_prior_scale_input.on_change("value", update_data)

use_volume_checkbox = CheckboxGroup(labels=["Use Volume as Regressor"], active=[0])
use_volume_checkbox.on_change("active", update_data)

# Plot
p = figure(title=f"{symbol} Price Forecast", x_axis_label="Date", x_axis_type="datetime", y_axis_label="Price", width=1600, tools=[hover, BoxZoomTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool(), BoxSelectTool(), TapTool(), CrosshairTool()])
p.line(x="ds", y="y", source=source_actual, legend_label="Actual Price", color="blue")
p.line(x="ds", y="yhat", source=source_forecast, legend_label="Forecasted Price", color="red")
p.legend.location = "top_left"

layout = column(row(symbol_select, interval_select, predict_period_input, seasonality_mode_select, changepoint_prior_scale_input, use_volume_checkbox), p)

curdoc().add_root(layout)

In [ ]:

prophet_df.head()
